<a href="https://colab.research.google.com/github/HajarahM/Mistral-7B/blob/main/Chat_with_MultiplePDFs_Mistral_7B_Instruct1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install langchain
%pip install torch
%pip install sentence_transformers
%pip install faiss-cpu
%pip install huggingface-hub
%pip install pypdf
%pip -q install accelerate
%pip install llama-cpp-python
%pip -q install git+https://github.com/huggingface/transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 47.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 53.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 33.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 92.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 77.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 32.6 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=36770cd83b183c2f5e8d5c0240b6d4a462a5dc9edfdb553

In [23]:
from langchain.chains import RetrievalQA
from langchain.embeddings import OllamaEmbeddings
from langchain.llms import Ollama
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.document_loaders import DirectoryLoader

In [24]:
#load pdf files
loader = DirectoryLoader("./data/")
data = loader.load()

In [25]:
print(data)

[Document(page_content='CHAPTER 4\n\nTHE ARBITRATION AND CONCILIATION ACT.\n\nArrangement of Sections.\n\nSection\n\nPART I—PRELIMINARY.\n\n1. 2.\n\nApplication. Interpretation.\n\nPART II—ARBITRATION.\n\n3. 4. Waiver of right to object. Stay of legal proceedings. 5. Interim measures by the court. 6. Death of a party. 7. Receipt of written communications. 8. Extent of court intervention. 9. 10. Determination of number of arbitrators. 11. Appointment of arbitrators. 12. Grounds for challenge. 13. Challenge procedure. 14. Failure or impossibility to act. 15. Termination of mandate and substitution of arbitrator. 16. Competence of arbitral tribunal to rule on its jurisdiction. 17. Power of arbitral tribunal. 18. Equal treatment of parties. 19. Determination of rules of procedure. 20. Place of arbitration. 21. Commencement of arbitral proceedings. 22. Language. 23. Statements of claim and defence. 24. Hearing and written submissions. 25. Default of a party. 26. Experts. 27. Court assistanc

In [26]:
#Step 05: Split the Extracted Data into Text Chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=20)

text_chunks = text_splitter.split_documents(data)


In [27]:
len(text_chunks)

55

In [29]:
#get the third chunk
text_chunks[9]

Document(page_content="506\n\n(2) Subrule (1) does not apply to any information, which would in any case be required to be disclosed in proceedings in the main suit or an application arising out of the suit.\n\n(3) A party to mediation under these rules shall not compel the mediator or employee, officer or representative of CADER to appear as a witness, consultant, or expert in any litigation or other proceedings related to the mediation.\n\n19. Adherence to guidelines for mediators. A person who conducts mediation under these Rules shall adhere to the guidelines for mediators set out in Schedule 2.\n\n20. Costs.\n\n(1) Each party to mediation shall bear its own costs of participation\n\nin the mediation under these Rules unless the parties agree otherwise.\n\n(2) For the avoidance of doubt adjournment costs provided for under rule 14 are not to be taken as costs of participation in mediation and the liability to pay adjournment costs is not affected by subrule (1).\n\n21. Monitoring a

In [30]:
#Step 06:Downlaod the Embeddings
embeddings = OllamaEmbeddings(model="mistral")


In [33]:
#Step 08: Create Embeddings for each of the Text Chunk
vector_store = FAISS.from_documents(text_chunks, embedding=embeddings)
print('saving embeddings to vector_store')
if os.path.exists(FAISS_vector_store):
    faiss_index=FAISS.load_local(FAISS_vector_store, embeddings)
    faiss_index.merge_from(vector_store)
    faiss_index.save_local(FAISS_vector_store)
else:
    vector_store.save_local(folder_path=FAISS_vector_store)

KeyboardInterrupt: 

In [ ]:

vectorstore.save_local("faiss_index")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#Import Model
llm = Ollama(
    model = "mistral",
    callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])
    )

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 


In [ ]:
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=vector_store.as_retriever(search_kwargs={"k": 2}))

In [ ]:
query = "What is in the arbitration act"

In [ ]:
qa.run(query)

' The linear regression model is a statistical method for modeling the relationship between a dependent variable and one or more independent variables, commonly referred to as predictors or explanatory variables. In this paper, the linear regression model is used as a starting point to derive estimators for the variance-covariance matrix of the errors in certain structural change models.'

In [ ]:
import sys

while True:
  user_input = input(f"Input Prompt: ")
  if user_input == 'exit':
    print('Exiting')
    sys.exit()
  if user_input == '':
    continue
  result = qa({'query': user_input})
  print(f"Answer: {result['result']}")

Llama.generate: prefix-match hit


Answer:  It is a statistical method used to analyze the linear relationship between two or more variables. In this case, it is used to estimate the coefficients of a linear regression model.


Llama.generate: prefix-match hit


Answer:  sandwich provides various functions for estimating the covariance matrix. The most commonly used ones are vcovHAC and vcovHC. These functions can take different weighting schemes, including kernel-based HAC estimation with automatic bandwidth selection based on weightsAndrews andbwAndrews. Other functions like weightsLumley provide different weighting schemes, such as truncated and smoothed weights, which are useful in certain applications. In econometric analyses, these functions are used to compute partial t-tests for assessing the significance of a parameter. The choice of estimator depends on the presence of heteroscedasticity and autocorrelation in the errors.
